In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import numpy as np

Using TensorFlow backend.


In [3]:
data = """The cat and her kittens
They put on their mittens,
To eat a Christmas pie.
The poor little kittens
They lost their mittens,
And then they began to cry.
O mother dear, we sadly fear
We cannot go to-day,
For we have lost our mittens."
"If it be so, ye shall not go,
For ye are naughty kittens."""

In [4]:
tokenizer = Tokenizer()

def dataset_preparation(data):
    corpus = data.lower().split("\n")
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len, total_words

def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len)) #Input Layer : Takes the sequence of words as input
    model.add(LSTM(150)) #LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.
    model.add(Dropout(0.1)) #Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer. It helps in preventing over fitting.
    model.add(Dense(total_words, activation='softmax')) #Output Layer : Computes the probability of the best possible next word as output
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(predictors, label, epochs=100, verbose=1)
    return model

def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=
                             max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [5]:
X, Y, msl, total_words = dataset_preparation(data)
model = create_model(X, Y, msl, total_words)

text = generate_text("cat and", 3, msl, model)
print(text)
#>>> "cat and her lost kittens"

Epoch 1/100
48/48 [==============================] - 1s 16ms/step - loss: 3.7619
Epoch 2/100
48/48 [==============================] - 0s 379us/step - loss: 3.7583
Epoch 3/100
48/48 [==============================] - 0s 375us/step - loss: 3.7551
Epoch 4/100
48/48 [==============================] - 0s 373us/step - loss: 3.7528
Epoch 5/100
48/48 [==============================] - 0s 357us/step - loss: 3.7492
Epoch 6/100
48/48 [==============================] - 0s 371us/step - loss: 3.7456
Epoch 7/100
48/48 [==============================] - 0s 356us/step - loss: 3.7409
Epoch 8/100
48/48 [==============================] - 0s 357us/step - loss: 3.7368
Epoch 9/100
48/48 [==============================] - 0s 378us/step - loss: 3.7314
Epoch 10/100
48/48 [==============================] - 0s 363us/step - loss: 3.7219
Epoch 11/100
48/48 [==============================] - 0s 359us/step - loss: 3.7150
Epoch 12/100
48/48 [==============================] - 0s 372us/step - loss: 3.7027
Epoch 13/100
4